In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

# Clustering

Até o momento exploramos uma area de aprendizagem não supervisionada: **redução de dimensionalidade** através de PCA.

Hoje olharemos outra classe de algoritmos que se enquadram neste tipo de aprendizagem: **algoritmos de clusterização**. Algoritmos de clusterização buscam aprender, a partir das variáveis observadas, uma categorização ótima dos pontos em grupos distintos.

# Conhecendo KMeans

## KMeans in 2-Dimensions

Vamos analisar um problema simples (em 2 variáveis) a partir de dados simulados.

In [ ]:
# from sklearn.datasets.samples_generator import make_blobs
from sklearn.datasets import make_blobs


X, y_true = make_blobs(n_samples=300, centers=4,
                       cluster_std=0.6, random_state=0)

tb_blob = pd.DataFrame(X)
tb_blob.columns = ['X1', 'X2']
sns.scatterplot(data = tb_blob, x = 'X1', y = 'X2');

A partir do gráfico acima podemos inferir que deveria ser possível separar os pontos, através destas duas variáveis, em 4 grupos distintos.

## O que são Centroides?

Os centroides são o centro de massa de cada grupo que vamos calcular.

No algoritmo **KMeans**, devemos especificar quantos grupos iremos buscar através do parametro **n_clusters**.

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
clu_fit = KMeans(n_clusters = 4)

In [ ]:
tb_blob.head()

Quando utilizamos o algoritmo **KMeans** na implatanção da Scikit-Learn, os principais resultados são armazenados nos atributos **kmeans.cluster_centers_** e **kmeans.labels_**

O atributo **kmeans.labels_** guarda as previsões dos agrupamentos para o dataset que utilizamos durante a etapa de aprendizagem. Podemos utilizar o método **kmeans.predict(X)** para obter a mesma variável categórica resultante.

In [ ]:
clu_fit = KMeans(n_clusters = 4)

In [ ]:
clu_fit.fit(tb_blob[['X1', 'X2']])

In [ ]:
tb_blob['pred_cluster'] = clu_fit.predict(tb_blob[['X1', 'X2']])
# Também poderíamos usar tb_blob['pred_cluster'] = clu_fit.labels_
tb_blob['pred_cluster'] = tb_blob['pred_cluster'].astype(str)

In [ ]:
tb_blob.head()

In [ ]:
centroids = pd.DataFrame(clu_fit.cluster_centers_)
centroids.columns = ['X1', 'X2']
centroids.head()

In [ ]:
print(clu_fit.cluster_centers_)
plt.figure(figsize=(8,8))
sns.scatterplot(data = tb_blob, x = 'X1', y = 'X2', hue = 'pred_cluster', s = 100);
sns.scatterplot(data = centroids, x = 'X1', y = 'X2', color = 'black', s = 250);

## Os passos de KMeans

### Primeira Iteração - Inicialização dos Centróides

Começamos inicializando um vetor de centroides aleatórios.

In [ ]:
p1 = (np.quantile(tb_blob["X1"], 0.25), np.quantile(tb_blob["X2"], 0.25))
p2 = (np.quantile(tb_blob["X1"], 0.25), np.quantile(tb_blob["X2"], 0.75))
p3 = (np.quantile(tb_blob["X1"], 0.75), np.quantile(tb_blob["X2"], 0.25))
p4 = (np.quantile(tb_blob["X1"], 0.75), np.quantile(tb_blob["X2"], 0.75))

tb_centroids = pd.DataFrame(
    [p1, p2, p3, p4], columns=["X1", "X2"], index=["p1", "p2", "p3", "p4"]
)
tb_centroids

In [ ]:
sns.scatterplot(data=tb_blob, x="X1", y="X2")
sns.scatterplot(data=tb_centroids, x="X1", y="X2", color="black", s=100)

### Primeira Iteração - Calculando Membros de Clusters
Agora, vamos atualizar a posição dos nossos centróides:

In [ ]:
for centroid in tb_centroids.iterrows():
    centroid_name = centroid[0]
    centroid_x1 = centroid[1]["X1"]
    centroid_x2 = centroid[1]["X2"]
    tb_blob[centroid_name] = (tb_blob["X1"] - centroid_x1) ** 2 + (tb_blob["X2"] - centroid_x2) ** 2

tb_blob.head()

In [ ]:
tb_blob["cluster"] = tb_blob[["p1", "p2", "p3", "p4"]].idxmin(1)
tb_blob.head()

In [ ]:
sns.scatterplot(data=tb_blob, x="X1", y="X2", hue="cluster")
sns.scatterplot(data=tb_centroids, x="X1", y="X2", color="black", s=100)

### Primeira Atualização - Recalculando Centróides

Agora, vamos utilizar os pontos de cada cluster estimados na etapa anterior para atualizar a posição de meus centróides:

In [ ]:
tb_centroids = tb_blob.groupby("cluster")[["X1", "X2"]].mean()
tb_centroids

In [ ]:
sns.scatterplot(data=tb_blob, x="X1", y="X2", hue="cluster")
sns.scatterplot(data=tb_centroids, x="X1", y="X2", color="black", s=100)

### Segunda Iteração - Calculando Membros de Clusters

A segunda iteração (e todas as subsequentes) repetem as duas últimas etapas da primeira iteração: primeiro atualizamos os membros de nossos clusters de acordo com os centróides calculados na última etapa...

In [ ]:
for centroid in tb_centroids.iterrows():
    centroid_name = centroid[0]
    centroid_x1 = centroid[1]["X1"]
    centroid_x2 = centroid[1]["X2"]
    tb_blob[centroid_name] = (tb_blob["X1"] - centroid_x1) ** 2 + (
        tb_blob["X2"] - centroid_x2
    ) ** 2
tb_blob["cluster"] = tb_blob[["p1", "p2", "p3", "p4"]].idxmin(1)
tb_blob.head()

In [ ]:
sns.scatterplot(data=tb_blob, x="X1", y="X2", hue="cluster")
sns.scatterplot(data=tb_centroids, x="X1", y="X2", color="black", s=100)

### Segunda Atualização - Recalculando Centróides

...depois recalculamos os centróides a partir dos membros de cada cluster.

In [ ]:
tb_centroids = tb_blob.groupby("cluster")[["X1", "X2"]].mean()
sns.scatterplot(data=tb_blob, x="X1", y="X2", hue="cluster")
sns.scatterplot(data=tb_centroids, x="X1", y="X2", color="black", s=100)

## Problemas do KMeans

Apesar de sua simplicidade e efetividade, o algoritmo k-Means apresenta um grande problema: como determinamos **k**? Se nossos espaço de features tem 2 dimensões, podemos fazer um gráfico, como fizemos até agora. A maior parte dos problemas reais apresenta mais dimensões: nesses casos utilizaremos algumas ferramentas para auxiliar nessa determinação.

Começaremos vendo as medidas de inérica e silhueta - que podemos utilizar para 'otimizar' o número de clusters.

### Selecionando K

In [ ]:
clu_fit = KMeans(n_clusters = 2, init = 'k-means++', n_init = 10)
clu_fit.fit(tb_blob[['X1', 'X2']])
tb_blob['pred_cluster'] = clu_fit.predict(tb_blob[['X1', 'X2']])
tb_blob['pred_cluster'] = tb_blob['pred_cluster'].astype(str)
centroids = pd.DataFrame(clu_fit.cluster_centers_)
centroids.columns = ['X1', 'X2']
plt.figure(figsize=(8,8))
sns.scatterplot(data = tb_blob, x = 'X1', y = 'X2', hue = 'pred_cluster', s = 100);
sns.scatterplot(data = centroids, x = 'X1', y = 'X2', color = 'black', s = 250);

In [ ]:
clu_fit = KMeans(n_clusters = 4)
clu_fit.fit(tb_blob[['X1', 'X2']])
tb_blob['pred_cluster'] = clu_fit.predict(tb_blob[['X1', 'X2']])
tb_blob['pred_cluster'] = tb_blob['pred_cluster'].astype(str)
centroids = pd.DataFrame(clu_fit.cluster_centers_)
centroids.columns = ['X1', 'X2']
plt.figure(figsize=(8,8))
sns.scatterplot(data = tb_blob, x = 'X1', y = 'X2', hue = 'pred_cluster', s = 100);
sns.scatterplot(data = centroids, x = 'X1', y = 'X2', color = 'black', s = 250);

#### Inércia

A inércia é uma medida da coesão interna de um cluster: ela mede o quão parecido os pontos de um dado grupo são entre si e é uma medida de distorção (quanto maior, piores os clusters). Ela é medida como a soma do quadrado da distância de cada ponto ao centroide de seu cluster.

https://towardsdatascience.com/clustering-how-to-find-hyperparameters-using-inertia-b0343c6fe819

In [ ]:
inertia_list = []

for i in range(2, 11):
    clu_fit = KMeans(n_clusters = i)
    clu_fit.fit(tb_blob[['X1', 'X2']])
    inertia_list.append(clu_fit.inertia_)
    
tb_inertia = pd.DataFrame({'k' : list(range(2, 11)), 'inertia' : inertia_list})
sns.lineplot(data = tb_inertia, x = 'k', y = 'inertia');

In [ ]:
from yellowbrick.cluster import KElbowVisualizer

In [ ]:
model = KMeans()
visualizer = KElbowVisualizer(model, k=(2,10))

visualizer.fit(tb_blob[['X1', 'X2']])
visualizer.show();

#### Silhueta

Embora a inércia seja uma ótima medida de quanto cada ponto pertence ao seu cluster ela não leva em conta o quão parecido cada ponto é com os outros clusters. A silhueta é uma medida associada à cada ponto do nosso conjunto de dados que mede a relação entre *o quão parecido cada ponto é com seu proprio cluster* e o *quão parecido cada ponto é com o cluster mais próximo ao qual ele não pertence*. Ela é definida como a diferença entre a distância média de um ponto para cada elemento de seu cluter e a distância de um ponto para cada elemento do cluster mais próximo (que não aquele ao qual ele pertence) normalizada pelo máximo entre essas duas distâncias.

Como ela é medida por ponto, podemos utiliza-la para avaliar quão bem *enquadrado* cada um está em seu cluster - os pontos com silhueta = 1 são os melhores enquadrados, os com silhueta = -1 os piores.

https://towardsdatascience.com/silhouette-method-better-than-elbow-method-to-find-optimal-clusters-378d62ff6891

In [ ]:
clu_fit = KMeans(n_clusters = 2)
clu_fit.fit(tb_blob[['X1', 'X2']])
tb_blob['pc_k2'] = clu_fit.predict(tb_blob[['X1', 'X2']])
tb_blob['pc_k2'] = tb_blob['pc_k2'].astype(str)

In [ ]:
clu_fit = KMeans(n_clusters = 10)
clu_fit.fit(tb_blob[['X1', 'X2']])
tb_blob['pc_k10'] = clu_fit.predict(tb_blob[['X1', 'X2']])
tb_blob['pc_k10'] = tb_blob['pc_k10'].astype(str)

In [ ]:
tb_blob.head()

In [ ]:
from sklearn.metrics import silhouette_samples

In [ ]:
tb_blob['sil_k2'] = silhouette_samples(tb_blob[['X1', 'X2']], tb_blob['pc_k2'])
tb_blob['sil_k10'] = silhouette_samples(tb_blob[['X1', 'X2']], tb_blob['pc_k10'])

In [ ]:
tb_blob.head()

In [ ]:
plt.figure(figsize=(8, 8))
sns.scatterplot(data=tb_blob,
                x='X1',
                y='X2',
                hue='pc_k2',
                size='sil_k2',
                sizes=(10, 150))

In [ ]:
plt.figure(figsize=(8, 8))
sns.scatterplot(data=tb_blob,
                x='X1',
                y='X2',
                hue='pc_k10',
                size='sil_k10',
                sizes=(10, 150))

In [ ]:
from sklearn.metrics import silhouette_score
sil_list = []

for i in range(2, 11):
    clu_fit = KMeans(n_clusters = i)
    clu_fit.fit(tb_blob[['X1', 'X2']])
    sil = silhouette_score(tb_blob[['X1', 'X2']], clu_fit.labels_)
    sil_list.append(sil)
    
tb_inertia = pd.DataFrame({'k' : list(range(2, 11)), 'silhouette' : sil_list})
sns.lineplot(data = tb_inertia, x = 'k', y = 'silhouette');

In [ ]:
from yellowbrick.cluster import SilhouetteVisualizer

model = KMeans(2, random_state=42)
visualizer = SilhouetteVisualizer(model, colors='yellowbrick')

visualizer.fit(tb_blob[['X1', 'X2']])
visualizer.show();

# Um exemplo prático

In [ ]:
tb_usarrests = pd.read_csv('data/USArrests.csv', index_col=0)
tb_usarrests.head()

In [ ]:
sns.pairplot(data = tb_usarrests)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

## Padronizar os dados

In [ ]:
scaler = StandardScaler().fit(tb_usarrests)
tb_std_usa = pd.DataFrame(scaler.transform(tb_usarrests), columns = tb_usarrests.columns)

In [ ]:
tb_std_usa.head()

## Como visualizar relações em mais do que 2 dimensões?

In [ ]:
pca_fit = PCA()
pca_fit.fit(tb_std_usa);
print(np.cumsum(pca_fit.explained_variance_ratio_))
tb_pca_usa = pd.DataFrame(pca_fit.transform(tb_std_usa),
                          columns = ['PC1', 'PC2', 'PC3', 'PC4'])

In [ ]:
plt.plot(pca_fit.explained_variance_ratio_)

In [ ]:
sns.scatterplot(data = tb_pca_usa, x = 'PC1', y = 'PC2')

## Escolhendo K

In [ ]:
model = KMeans()
visualizer = KElbowVisualizer(model, k=(2,10))

visualizer.fit(tb_pca_usa)
visualizer.show();

In [ ]:
sil_list = []

for i in range(2, 11):
    clu_fit = KMeans(n_clusters = i)
    clu_fit.fit(tb_pca_usa)
    sil = silhouette_score(tb_pca_usa, clu_fit.labels_)
    sil_list.append(sil)
    
tb_inertia = pd.DataFrame({'k' : list(range(2, 11)), 'silhouette' : sil_list})
sns.lineplot(data = tb_inertia, x = 'k', y = 'silhouette');

In [ ]:
from yellowbrick.cluster import SilhouetteVisualizer

model = KMeans(4)
visualizer = SilhouetteVisualizer(model, colors='yellowbrick')

visualizer.fit(tb_pca_usa)
visualizer.show();

## Analisando os resultados finais

In [ ]:
clu_fit_final = KMeans(4)
clu_fit_final.fit(tb_pca_usa)
tb_pca_usa['cluster'] = clu_fit_final.labels_
tb_pca_usa['cluster'] = tb_pca_usa['cluster'].astype(str)
tb_usarrests['cluster'] = clu_fit_final.labels_
tb_usarrests['cluster'] = tb_usarrests['cluster'].astype(str)

In [ ]:
sns.scatterplot(data = tb_pca_usa, x = 'PC1', y = 'PC2', hue = 'cluster', s = 100);

In [ ]:
sns.boxplot(data = tb_usarrests, x = 'cluster', y = 'Murder');

In [ ]:
sns.boxplot(data = tb_usarrests, x = 'cluster', y = 'Rape');

In [ ]:
sns.boxplot(data = tb_usarrests, x = 'cluster', y = 'Assault');

In [ ]:
sns.boxplot(data = tb_usarrests, x = 'cluster', y = 'UrbanPop');

In [ ]:
tb_usarrests[tb_usarrests['cluster'] == '3']